In [2]:
import spacy

nlp = spacy.blank("en")
nlp.add_pipe(
    "llm",
    config={
        "task": {
            "@llm_tasks": "spacy.NER.v2",
            "labels": ["PERSON", "ORGANISATION", "LOCATION", "PHONE NUMBER"]
        },
        "model": {
            "@llm_models": "spacy.GPT-3-5.v1",
        },
    },
)
nlp.initialize()
doc = nlp("Jack and Jill rode up the hill in Les Deux Alpes")
print([(ent.text, ent.label_) for ent in doc.ents])


[('Jack', 'PERSON'), ('Jill', 'PERSON'), ('Les Deux Alpes', 'LOCATION')]


In [45]:
# https://sourajit16-02-93.medium.com/zero-shot-named-entity-recognition-using-openai-chatgpt-api-46738191f375
fileOpen = open("phone_numbers_examples.txt", "r")
data = fileOpen.read().split("\n")
data = data[2:]
data = ",".join(list(filter(lambda x: x.strip(), data)))
# print(data)

import os 

SYSTEM_PROMPT = "You are a smart and intelligent regex Generator system. I will provide you the examples of text you need to find a regex expression for and the output format. Do not respond with text other than that of the output format."

USER_PROMPT_1 = "Are you clear about your role?"

ASSISTANT_PROMPT_1 = "Sure, I'm ready to help you with your regex task. Please provide me with the necessary information to get started."

GUIDELINES_PROMPT = "Examples:\n" + data + "\n" + "Output Format:\n" + "REGEX: [regex expression]"

import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

def openai_chat_completion_response(final_prompt):
  response = openai.ChatCompletion.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": USER_PROMPT_1},
                    {"role": "assistant", "content": ASSISTANT_PROMPT_1},
                    {"role": "user", "content": final_prompt}
                ]
            )

  return response['choices'][0]['message']['content'].strip(" \n")

# my_sentence = "Hanyu Pinyin worked under Christopher in the same project for 2 years and the project started on 24-06-2018."
# GUIDELINES_PROMPT = GUIDELINES_PROMPT.format(my_sentence)
ners = openai_chat_completion_response(GUIDELINES_PROMPT)
print(ners)

REGEX: \b\d{3}-\d{3}-\d{4}\b|\(\d{3}\) \d{3}-\d{4}\b|\d{4} \d{5}\b|\d{3} \d{3} \d{4}\b|\d{10}\b|\d{3}-\d{4}


In [55]:
import re
h = re.findall(ners, "123-1234 124-2312")
print(h)

['123-1234', '124-2312']
